## spark udf 테스트

In [4]:
import pandas as pd
import numpy as np
import re
import json
from konlpy.tag import Okt
import nltk # 자연어 처리 패키지
from gensim.models import FastText
import fasttext
from nltk.tokenize import word_tokenize

from tqdm import tqdm # 작업 진행률 표시하는 라이브러리 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

In [2]:
pip freeze

absl-py==0.15.0
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
asttokens==2.0.5
astunparse==1.6.3
attrs==21.4.0
autovizwidget==0.19.2
backcall==0.2.0
beautifulsoup4==4.11.1
bleach==5.0.0
boto3==1.23.4
botocore==1.26.4
brotlipy @ file:///home/conda/feedstock_root/build_artifacts/brotlipy_1648854164373/work
cachetools==5.1.0
certifi==2021.10.8
cffi @ file:///home/conda/feedstock_root/build_artifacts/cffi_1636046055389/work
charset-normalizer @ file:///home/conda/feedstock_root/build_artifacts/charset-normalizer_1644853463426/work
click==8.1.3
colorama @ file:///home/conda/feedstock_root/build_artifacts/colorama_1602866480661/work
conda==4.12.0
conda-package-handling @ file:///home/conda/feedstock_root/build_artifacts/conda-package-handling_1649385069444/work
cryptography @ file:///tmp/build/80754af9/cryptography_1652101588893/work
Cython==0.29.30
debugpy==1.6.0
decorator==5.1.1
defusedxml==0.7.1
entrypoints==0.4
executing==0.8.3
fastjsonschema==2.15.3
fasttext==0.9.2
filelock==3.7.0
fl

In [3]:
import gensim

In [6]:
import pandas as pd
import numpy as np
import re
import json
from konlpy.tag import Okt
import nltk # 자연어 처리 패키지
from gensim.models import FastText
from nltk.tokenize import word_tokenize

from tqdm import tqdm # 작업 진행률 표시하는 라이브러리 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import warnings

warnings.filterwarnings(action='ignore')


###### 모델 불러오기 ######

lstm_model = load_model('./fasttext_LSTM1.h5')
embedded_model = FastText.load("./festtext_embedded_1.model")

###########################

# okt = Okt()
tokenizer = Tokenizer()
# stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','을','으로','자','에','와','한','하다','부터']
# max_len = 40
# vocab_size = 24832
# tokenizer = Tokenizer(vocab_size)

###### json 파일 불러오기 ######

# with open('./Fast-text/fasttext_vectorized_1.json' , encoding='UTF8') as json_file:
#     vocab = json.load(json_file)
# #     tokenizer.word_index = vocab

################################    

# 자모분리

CHOSUNGS = [u'ㄱ',u'ㄲ',u'ㄴ',u'ㄷ',u'ㄸ',u'ㄹ',u'ㅁ',u'ㅂ',u'ㅃ',u'ㅅ',u'ㅆ',u'ㅇ',u'ㅈ',u'ㅉ',u'ㅊ',u'ㅋ',u'ㅌ',u'ㅍ',u'ㅎ']
JOONGSUNGS = [u'ㅏ',u'ㅐ',u'ㅑ',u'ㅒ',u'ㅓ',u'ㅔ',u'ㅕ',u'ㅖ',u'ㅗ',u'ㅘ',u'ㅙ',u'ㅚ',u'ㅛ',u'ㅜ',u'ㅝ',u'ㅞ',u'ㅟ',u'ㅠ',u'ㅡ',u'ㅢ',u'ㅣ']
JONGSUNGS = [u'_',u'ㄱ',u'ㄲ',u'ㄳ',u'ㄴ',u'ㄵ',u'ㄶ',u'ㄷ',u'ㄹ',u'ㄺ',u'ㄻ',u'ㄼ',u'ㄽ',u'ㄾ',u'ㄿ',u'ㅀ',u'ㅁ',u'ㅂ',u'ㅄ',u'ㅅ',u'ㅆ',u'ㅇ',u'ㅈ',u'ㅊ',u'ㅋ',u'ㅌ',u'ㅍ',u'ㅎ']
TOTAL = CHOSUNGS + JOONGSUNGS + JONGSUNGS

# 자모분리
def jamo_split(word, end_char="_"):
    
    result = []
    
    for char in word:
        
        character_code = ord(char)
        
        if 0xD7A3 < character_code or character_code < 0xAC00:
            result.append(char)
            continue

        chosung_index = int((((character_code - 0xAC00) / 28) / 21) % 19)
        joongsung_index = int(((character_code - 0xAC00) / 28) % 21)
        jongsung_index = int((character_code - 0xAC00) % 28)
        
        chosung = CHOSUNGS[chosung_index]
        joongsung = JOONGSUNGS[joongsung_index]
        jongsung = JONGSUNGS[jongsung_index]
        
        # 종성 범위 밖에 있는 것들은 end_char로 메꿔준다.
        if jongsung_index == 0:
            jongsung = end_char
        
        result.append(chosung)
        result.append(joongsung)
        result.append(jongsung)

    return "".join(result)    
    
################################
def test_result(s, embedded_model, lstm_model):
    test_word = jamo_split(s)
    test_word_split = test_word.split()
    fast_vec = []
    for index in range(len(test_word_split)):
        if index < len(test_word_split):
            fast_vec.append(embedded_model[test_word_split[index]])
        else:
            fast_vec.append(np.array([0]*100))
    fast_vec = np.array(fast_vec)
    fast_vec=fast_vec.reshape(1, fast_vec.shape[0], fast_vec.shape[1])
    # 학습 데이터와 마찬가지로 3차원으로 크기 조절
    test_pre = lstm_model.predict_classes([fast_vec]) # 비속어 판별
    if test_pre[0][0] == 1:
        return '{0} 비속어 포함!!'.format(test_pre)
    else:
        return '{0} 비속어 없어요 ^.^'.format(test_pre)
    

FileNotFoundError: [Errno 2] No such file or directory: './festtext_embedded_1.model.wv.vectors_ngrams.npy'

In [259]:
    
################################
def test_result(s, embedded_model, lstm_model):
    test_word = jamo_split(s)
    test_word_split = test_word.split()
    fast_vec = []
    for index in range(len(test_word_split)):
        if index < len(test_word_split):
            fast_vec.append(embedded_model[test_word_split[index]])
        else:
            fast_vec.append(np.array([0]*100))
    fast_vec = np.array(fast_vec)
    fast_vec=fast_vec.reshape(1, fast_vec.shape[0], fast_vec.shape[1])
    # 학습 데이터와 마찬가지로 3차원으로 크기 조절
    test_pre = lstm_model.predict_classes([fast_vec]) # 비속어 판별
    if test_pre[0][0] == 1:
        return '{0} 비속어 포함!!'.format(test_pre[0][0])
    else:
        return '{0} 비속어 없어요 ^.^'.format(test_pre[0][0])


In [260]:
test_result("개짜증나네", embedded_model, lstm_model)

'1 비속어 포함!!'

In [261]:
test_result("미친새끼", embedded_model, lstm_model)

'1 비속어 포함!!'

In [262]:
test_result("초콜릿 사러 편의점 갈라고", embedded_model, lstm_model)

'1 비속어 포함!!'

In [263]:
test_result("존나 재밌다!!", embedded_model, lstm_model)

'0 비속어 없어요 ^.^'

In [264]:
test_result("존나 빡치네", embedded_model, lstm_model)

'1 비속어 포함!!'

In [265]:
test_result("이번주에 친구 보러 가", embedded_model, lstm_model)

'0 비속어 없어요 ^.^'

In [146]:
test_result("롤 재미있겠지? 오늘 저녁에 나한테 한번 배워볼래?", embedded_model, lstm_model)

'[[0]] 비속어 없어요 ^.^'

In [150]:
test_result("파란색깔 옷을 살라구요~ 지금 쇼핑몰 먼저 보고있어요", embedded_model, lstm_model)

'[[0]] 비속어 없어요 ^.^'

In [151]:
test_result("맛있겠다 하나 사고싶은데 존나 비쌈 ㅠㅠ", embedded_model, lstm_model)

'[[1]] 비속어 포함!!'

In [152]:
test_result("와 개싸다 하나 득템각", embedded_model, lstm_model)

'[[1]] 비속어 포함!!'

In [153]:
test_result("와 싸다 하나 득템각", embedded_model, lstm_model)

'[[1]] 비속어 포함!!'

In [266]:
test_result("와 싸네 하나 득템각", embedded_model, lstm_model)

'1 비속어 포함!!'

In [155]:
test_result("와 하나 득템각", embedded_model, lstm_model)

'[[1]] 비속어 포함!!'

In [156]:
test_result("득템각", embedded_model, lstm_model)

'[[0]] 비속어 없어요 ^.^'

In [158]:
test_result("하나 득템각", embedded_model, lstm_model)

'[[0]] 비속어 없어요 ^.^'

In [159]:
test_result("와 싸다", embedded_model, lstm_model)

'[[1]] 비속어 포함!!'

In [161]:
test_result("싸다", embedded_model, lstm_model)

'[[0]] 비속어 없어요 ^.^'

In [162]:
test_result("와", embedded_model, lstm_model)

'[[0]] 비속어 없어요 ^.^'

In [166]:
test_result("ㅁㅊ", embedded_model, lstm_model)

'[[0]] 비속어 없어요 ^.^'

In [180]:
test_result("야동", embedded_model, lstm_model)

'[[1]] 비속어 포함!!'

In [267]:
test_result("이뿌당ㅠㅠ", embedded_model, lstm_model)

'1 비속어 포함!!'

In [268]:
test_result('맛있겠지?',  embedded_model, lstm_model)

'1 비속어 포함!!'

In [278]:
test_result('히히히',  embedded_model, lstm_model)

'0 비속어 없어요 ^.^'

## 크롤링 데이터 넣어서 보기

In [181]:
df = pd.read_csv('./dataset/153cm_키작녀_시니럽의_봄_여름_코디🎨.csv')
df.head()

,id,comment,program,time,love,play
0,마리아,좋을호 맞넹,153cm_키작녀_시니럽의_봄_여름_코디🎨,2022-05-02_09:54:04,704,"5,400"
1,lov,^^,153cm_키작녀_시니럽의_봄_여름_코디🎨,2022-05-02_09:54:04,704,"5,400"
2,마리아,ㅎㅎ,153cm_키작녀_시니럽의_봄_여름_코디🎨,2022-05-02_09:54:04,704,"5,400"
3,마리아,👏 👏,153cm_키작녀_시니럽의_봄_여름_코디🎨,2022-05-02_09:54:04,704,"5,400"
4,lov,이옷이 좀 더 날씬해 보여요,153cm_키작녀_시니럽의_봄_여름_코디🎨,2022-05-02_09:54:04,704,"5,400"


In [182]:
df = df.drop(columns = ['program', 'time', 'love', 'play'], axis=1)

In [183]:
df.sample(5)

,id,comment
14,쭈니,놀러가구싶당 ㅜㅜ
25,치킨,나왓다 언니최애
2,마리아,ㅎㅎ
7,lov,가단이 어떻게 되나영?
44,치킨,와 가격도 좋네용


In [200]:
# json-> df
# df= 네이버 csv

df["swear"]= df["comment"].map(lambda x: test_result(x, embedded_model, lstm_model))

In [211]:
df.sample(10)

,id,comment,swear
54,쭈니,와 레알 머리숱 부럽,[[1]] 비속어 포함!!
48,락락,알프스 소녀느낌,[[0]] 비속어 없어요 ^.^
26,락락,굿굿,[[0]] 비속어 없어요 ^.^
44,치킨,와 가격도 좋네용,[[0]] 비속어 없어요 ^.^
15,쭈니,언니 옷입고 ㅠㅠ,[[0]] 비속어 없어요 ^.^
41,배,아래란다,[[0]] 비속어 없어요 ^.^
13,nis,다른티셔츠도 입어바주세요,[[0]] 비속어 없어요 ^.^
22,쭈니,대박 시크,[[0]] 비속어 없어요 ^.^
42,배,모자가 포인트,[[0]] 비속어 없어요 ^.^
4,lov,이옷이 좀 더 날씬해 보여요,[[0]] 비속어 없어요 ^.^


### 함수변경

In [212]:
# def test_result(s, embedded_model, lstm_model):
#     test_word = jamo_split(s)
#     test_word_split = test_word.split()
#     fast_vec = []
#     for index in range(len(test_word_split)):
#         if index < len(test_word_split):
#             fast_vec.append(embedded_model[test_word_split[index]])
#         else:
#             fast_vec.append(np.array([0]*100))
#     fast_vec = np.array(fast_vec)
#     fast_vec=fast_vec.reshape(1, fast_vec.shape[0], fast_vec.shape[1])
#     # 학습 데이터와 마찬가지로 3차원으로 크기 조절
#     test_pre = lstm_model.predict_classes([fast_vec]) # 비속어 판별
#     if test_pre[0][0] == 0:
#         print(s)
#         return "비속어가 포함되어 있습니다."
#     else:
#         print(s)
#         return "비속어가 포함되어 있지 않습니다."
    
    
def test_result2(s, embedded_model, lstm_model):
    s = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', str(s))
    if s == "" or s.isspace():
        return "비속어가 포함되어있지 않습니다"
    else:
        test_word = jamo_split(s)
        test_word_split = test_word.split()
        fast_vec = []
        for index in range(len(test_word_split)):
            if index < len(test_word_split):
                fast_vec.append(embedded_model[test_word_split[index]])
            else:
                fast_vec.append(np.array([0]*100))
        fast_vec = np.array(fast_vec)
        fast_vec=fast_vec.reshape(1, fast_vec.shape[0], fast_vec.shape[1])
    # 학습 데이터와 마찬가지로 3차원으로 크기 조절
        test_pre = lstm_model.predict_classes([fast_vec]) # 비속어 판별
        if test_pre[0][0] == 1:
            return '{0} 비속어 포함!!'.format(test_pre)
        else:
            return '{0} 비속어 없어요 ^.^'.format(test_pre)

In [213]:
df["swearr"]= df["comment"].map(lambda x: test_result2(x, embedded_model, lstm_model))

In [224]:
df.sample(10)

,id,comment,swear,swearr
43,배,ㅋㅋㅋㅋㅋㅋ,[[0]] 비속어 없어요 ^.^,[[0]] 비속어 없어요 ^.^
54,쭈니,와 레알 머리숱 부럽,[[1]] 비속어 포함!!,[[1]] 비속어 포함!!
48,락락,알프스 소녀느낌,[[0]] 비속어 없어요 ^.^,[[0]] 비속어 없어요 ^.^
36,쭈니,하얀 모자에,[[0]] 비속어 없어요 ^.^,[[0]] 비속어 없어요 ^.^
27,노뚜호,👋,[[0]] 비속어 없어요 ^.^,비속어가 포함되어있지 않습니다
47,쭈니,러블리해,[[0]] 비속어 없어요 ^.^,[[0]] 비속어 없어요 ^.^
45,치킨,이뿌당 진짜,[[1]] 비속어 포함!!,[[1]] 비속어 포함!!
55,킴구사이,산뜻해요,[[0]] 비속어 없어요 ^.^,[[0]] 비속어 없어요 ^.^
52,배,머리숱 진짜 볼떄마다 부럽,[[0]] 비속어 없어요 ^.^,[[0]] 비속어 없어요 ^.^
53,배,ㄹㄹㄹ,[[0]] 비속어 없어요 ^.^,[[0]] 비속어 없어요 ^.^


In [16]:
test_result2("마라탕 맛있겠지?", embedded_model, lstm_model)

마라탕 맛있겠지


'비속어가 포함되어 있습니다.'

In [17]:
test_result("마라탕 맛있겠지?", embedded_model, lstm_model)

마라탕 맛있겠지?


'비속어가 포함되어 있지 않습니다.'

In [240]:
def test_result3(s, embedded_model, lstm_model):
    test_word = jamo_split(s)
    test_word_split = test_word.split()
    fast_vec = []
    for index in range(len(test_word_split)):
        if index < len(test_word_split):
            fast_vec.append(embedded_model[test_word_split[index]])
        else:
            fast_vec.append(np.array([0]*100))
    fast_vec = np.array(fast_vec)
    fast_vec=fast_vec.reshape(1, fast_vec.shape[0], fast_vec.shape[1])
    # 학습 데이터와 마찬가지로 3차원으로 크기 조절
    test_pre = lstm_model.predict_classes([fast_vec]) # 비속어 판별
    if test_pre[0][0] == 1:
        return "비속어가 포함되어 있습니다."
    else: 
        return "비속어가 포함되어 있지 않습니다."


In [241]:
test_result3("맛있겠지?", embedded_model, lstm_model)

'비속어가 포함되어 있습니다.'

In [242]:
test_result3("앞으로 자주올게요", embedded_model, lstm_model)

'비속어가 포함되어 있지 않습니다.'

In [245]:
df["swearrr"]= df["comment"].map(lambda x: test_result3(x, embedded_model, lstm_model))

In [246]:
df.sample(10)

,id,comment,swear,swearr,swearrr
43,배,ㅋㅋㅋㅋㅋㅋ,[[0]] 비속어 없어요 ^.^,[[0]] 비속어 없어요 ^.^,비속어가 포함되어 있지 않습니다.
13,nis,다른티셔츠도 입어바주세요,[[0]] 비속어 없어요 ^.^,[[0]] 비속어 없어요 ^.^,비속어가 포함되어 있지 않습니다.
32,락락,진짜 저런 손재주 없으니 나는 산다 .,[[1]] 비속어 포함!!,[[1]] 비속어 포함!!,비속어가 포함되어 있습니다.
30,치킨,손뜨개같앙,[[1]] 비속어 포함!!,[[1]] 비속어 포함!!,비속어가 포함되어 있습니다.
26,락락,굿굿,[[0]] 비속어 없어요 ^.^,[[0]] 비속어 없어요 ^.^,비속어가 포함되어 있지 않습니다.
41,배,아래란다,[[0]] 비속어 없어요 ^.^,[[0]] 비속어 없어요 ^.^,비속어가 포함되어 있지 않습니다.
7,lov,가단이 어떻게 되나영?,[[0]] 비속어 없어요 ^.^,[[0]] 비속어 없어요 ^.^,비속어가 포함되어 있지 않습니다.
27,노뚜호,👋,[[0]] 비속어 없어요 ^.^,비속어가 포함되어있지 않습니다,비속어가 포함되어 있지 않습니다.
23,쭈니,아니 저런 룩에 저 러블리한 모자가,[[1]] 비속어 포함!!,[[1]] 비속어 포함!!,비속어가 포함되어 있습니다.
39,배,대박,[[0]] 비속어 없어요 ^.^,[[0]] 비속어 없어요 ^.^,비속어가 포함되어 있지 않습니다.


In [244]:
def test_result4(s, embedded_model, lstm_model):
    test_word = jamo_split(s)
    test_word_split = test_word.split()
    fast_vec = []
    for index in range(len(test_word_split)):
        if index < len(test_word_split):
            fast_vec.append(embedded_model[test_word_split[index]])
        else:
            fast_vec.append(np.array([0]*100))
    fast_vec = np.array(fast_vec)
    fast_vec=fast_vec.reshape(1, fast_vec.shape[0], fast_vec.shape[1])
    # 학습 데이터와 마찬가지로 3차원으로 크기 조절
    test_pre = lstm_model.predict_classes([fast_vec]) # 비속어 판별
    if test_pre[0][0] == 0:
        return "비속어가 포함되어 있지않습니다."
    else:
        return "비속어가 포함되어 있습니다."


In [247]:
df["swearrrr"]= df["comment"].map(lambda x: test_result4(x, embedded_model, lstm_model))

In [250]:
df.sample(10)

,id,comment,swear,swearr,swearrr,swearrrr
43,배,ㅋㅋㅋㅋㅋㅋ,[[0]] 비속어 없어요 ^.^,[[0]] 비속어 없어요 ^.^,비속어가 포함되어 있지 않습니다.,비속어가 포함되어 있지않습니다.
56,점녀,키작녀에요 여기를 이제야 알다니 앞으로 자주 올게요,[[0]] 비속어 없어요 ^.^,[[0]] 비속어 없어요 ^.^,비속어가 포함되어 있지 않습니다.,비속어가 포함되어 있지않습니다.
41,배,아래란다,[[0]] 비속어 없어요 ^.^,[[0]] 비속어 없어요 ^.^,비속어가 포함되어 있지 않습니다.,비속어가 포함되어 있지않습니다.
49,락락,ㅎㅎㅎ,[[0]] 비속어 없어요 ^.^,[[0]] 비속어 없어요 ^.^,비속어가 포함되어 있지 않습니다.,비속어가 포함되어 있지않습니다.
1,lov,^^,[[0]] 비속어 없어요 ^.^,비속어가 포함되어있지 않습니다,비속어가 포함되어 있지 않습니다.,비속어가 포함되어 있지않습니다.
19,lov,44 엄청 날씬하시네요,[[0]] 비속어 없어요 ^.^,[[0]] 비속어 없어요 ^.^,비속어가 포함되어 있지 않습니다.,비속어가 포함되어 있지않습니다.
18,lov,언니 사이즈가 55시죠?,[[0]] 비속어 없어요 ^.^,[[0]] 비속어 없어요 ^.^,비속어가 포함되어 있지 않습니다.,비속어가 포함되어 있지않습니다.
26,락락,굿굿,[[0]] 비속어 없어요 ^.^,[[0]] 비속어 없어요 ^.^,비속어가 포함되어 있지 않습니다.,비속어가 포함되어 있지않습니다.
14,쭈니,놀러가구싶당 ㅜㅜ,[[1]] 비속어 포함!!,[[1]] 비속어 포함!!,비속어가 포함되어 있습니다.,비속어가 포함되어 있습니다.
53,배,ㄹㄹㄹ,[[0]] 비속어 없어요 ^.^,[[0]] 비속어 없어요 ^.^,비속어가 포함되어 있지 않습니다.,비속어가 포함되어 있지않습니다.


In [251]:
df2 = pd.read_csv('./dataset/2022-05-11.csv')
df2 = df2.head(50)

In [252]:
df2["swear score"]= df2["comment"].map(lambda x: test_result(x, embedded_model, lstm_model))
df2

,comment,swear score
0,동훈이 괴롭히지말고 쿨하게 가 거기 ㅋㅋㅋ,[[0]] 비속어 없어요 ^.^
1,병,[[1]] 비속어 포함!!
2,신,[[1]] 비속어 포함!!
3,아,[[0]] 비속어 없어요 ^.^
4,철책 짤라서 한반도 모양 십자가 만들었는데 이상하더라 느낌이 싸해 ㅠㅠㅠ,[[1]] 비속어 포함!!
5,누가보면 미사일 한두개날라오는줄알겠네 사드는 미군레이더감시망이지 그이상 그이하도 아...,[[1]] 비속어 포함!!
6,"전자파, 암 그런건 석열이가 알아서 하라그러고 ㅋ",[[1]] 비속어 포함!!
7,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 행복시티 그냥 개인적을 차단할거니까 니글 안보인다,[[0]] 비속어 없어요 ^.^
8,청문회는 끝난건가 민주당 웃음좀 주지 ㅋㅋ,[[1]] 비속어 포함!!
9,그니까 핵무기가 필요하지 우리만 맞냐,[[1]] 비속어 포함!!
